# Image Classification with pretrained CNN
### Using features from https://github.com/fchollet/deep-learning-models
    VGG16, VGG19, ResNet50, Inception v3 and CRNN for music tagging in Keras under MIT license
    
## Import
* Note: keras.metrics' fmeasure, precision and recall functions are newer than the last keras update, you may need to reload it manually(2016.11.05)

>  wget https://raw.githubusercontent.com/fchollet/keras/master/keras/metrics.py

>  sudo cp metrics.py /usr/local/lib/python2.7/dist-packages/keras/
    

In [49]:
from keras.preprocessing import image
from keras.metrics import binary_accuracy, fmeasure, precision, recall
from keras.optimizers import SGD,adam

from models.imagenet_utils import decode_predictions, preprocess_input
from models.vgg16 import VGG16

import urllib
import sys

import numpy as np

## Load VGG16

#### VGG16 conception: last convolutional block and fully-connected trainable
Note: The loaded model use 3 dense layers in the last block
<img src="assets/vgg16.png" width="350">
<em>image: https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html<em>

In [2]:
model = VGG16(weights='imagenet')

K.image_dim_ordering: th


## Preprocess images (ft. from GOID)
* Google Open Image Dataset: https://github.com/openimages/dataset
* [Preprocess Dataset](collect_data_info.py) URL-label pairs for the labels given in [this file](assets/dict.csv)

In [3]:
train_data_URL = []
with open("assets/train_url_labels.csv") as f:
    for line in f:
        next_line = line.split(",")
        train_data_URL.append([next_line[0],next_line[1].rstrip()])

In [4]:
len(train_data_URL)

8093

Generating (1:1000) np array from label (imagenet 1000 labels set)

In [5]:
imgnet_dict = {}
with open('assets/dict.csv') as f:
    for line in f:
        next_line = line.split('\t')
        imgnet_dict[next_line[1]]=int(next_line[2].rstrip())

#Better solution if we store the probability from the GOID and feed with that instead of 1.0
def generate_Y(label):
    a = np.zeros(1000, dtype="float32")
    a[imgnet_dict[label]]=1.0
    return a

### Loading training data
This may take several minutes

In [6]:
train_data = []
layers_num = {}
_end = len(train_data_URL)
for i in range(0,_end):
    if(layers_num.get(train_data_URL[i][1])!=None):
        layers_num[train_data_URL[i][1]]+=1
    else:
        layers_num[train_data_URL[i][1]]=1
    num = layers_num[train_data_URL[i][1]]
    if num<=30 :
        try:
            img_path = urllib.request.urlopen(train_data_URL[i][0])
            img = image.load_img(img_path, target_size=(224,224))
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            train_data.append([x,train_data_URL[i][1]])
            if(np.floor(i/_end*100)>np.floor((i-1)/_end*100)):
                sys.stdout.write(">")
        except ValueError:
            pass

>>>>>

### Loading validation data

In [7]:
validation_data_URL = []
with open("assets/validation_url_labels.csv") as f:
    for line in f:
        next_line = line.split(",")
        validation_data_URL.append([next_line[0],next_line[1].rstrip()])
        

In [8]:
validation_data = []
layers_num = {}
_end = len(validation_data_URL)
for i in range(0,_end):
    if(layers_num.get(validation_data_URL[i][1])!=None):
        layers_num[validation_data_URL[i][1]]+=1
    else:
        layers_num[validation_data_URL[i][1]]=1
    num = layers_num[validation_data_URL[i][1]]
    if num<=20 :
        try:
            img_path = urllib.request.urlopen(validation_data_URL[i][0])
            img = image.load_img(img_path, target_size=(224,224))
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            validation_data.append([x,validation_data_URL[i][1]])
            if(np.floor(i/_end*100)>np.floor((i-1)/_end*100)):
                sys.stdout.write(">")
        except ValueError:
            pass

>>>>>>>>>>>>>>>>>>>>

Cut test data from validation

In [9]:
valid_test_rate = 0.75
test_data = validation_data[round(valid_test_rate*len(validation_data)):]
validation_data = validation_data[:round(valid_test_rate*len(validation_data))]

Training-validation-test: 30-15-5

## Training
Only training fully connected first, later on train all.

In [72]:
#freeze layers
for layer in model.layers[:19]:
    layer.trainable = False
    print(layer)
print("")
for layer in model.layers[19:]:
    layer.trainable = True
    print(layer)

In [73]:
opt = adam(lr=0.5)
model.compile(loss='categorical_crossentropy', optimizer=opt,
metrics=['binary_accuracy', 'fmeasure', 'precision', 'recall'])

Solving input shape missmatches

In [12]:
tr_sets_tmp = []
tr_vals_tmp = []
for line in train_data:
    tr_sets_tmp.append(line[0][0])
    tr_vals_tmp.append(generate_Y(line[1]))
val_sets_tmp = []
val_vals_tmp = []
for line in validation_data:
    val_sets_tmp.append(line[0][0])
    val_vals_tmp.append(generate_Y(line[1]))
test_sets = []
test_vals = []
for line in test_data:
    test_sets.append(line[0][0])
    test_vals.append(generate_Y(line[1]))

In [13]:
tr_sets = np.array(tr_sets_tmp,  dtype= "float32")
tr_vals = np.array(tr_vals_tmp)
val_sets = np.array(val_sets_tmp,  dtype= "float32")
val_vals = np.array(val_vals_tmp)

In [14]:
print(np.shape(tr_sets))
print(np.shape(tr_vals))
print(np.shape(val_sets))
print(np.shape(val_vals))

(266, 3, 224, 224)
(266, 1000)
(141, 3, 224, 224)
(141, 1000)


In [74]:
model.fit(tr_sets,tr_vals,
          nb_epoch=20,
          verbose=1,
          batch_size = 32,
          validation_data=(val_sets, val_vals),
         shuffle= True)

Train on 266 samples, validate on 141 samples
Epoch 1/20
266/266 [==============================] - 118s - loss: 14.4820 - binary_accuracy: 0.9982 - fmeasure: 0.1015 - precision: 0.1015 - recall: 0.1015 - val_loss: 14.6320 - val_binary_accuracy: 0.9982 - val_fmeasure: 0.0922 - val_precision: 0.0922 - val_recall: 0.0922
Epoch 2/20
266/266 [==============================] - 118s - loss: 14.4820 - binary_accuracy: 0.9982 - fmeasure: 0.1015 - precision: 0.1015 - recall: 0.1015 - val_loss: 14.6320 - val_binary_accuracy: 0.9982 - val_fmeasure: 0.0922 - val_precision: 0.0922 - val_recall: 0.0922
Epoch 3/20
266/266 [==============================] - 118s - loss: 14.4820 - binary_accuracy: 0.9982 - fmeasure: 0.1015 - precision: 0.1015 - recall: 0.1015 - val_loss: 14.6320 - val_binary_accuracy: 0.9982 - val_fmeasure: 0.0922 - val_precision: 0.0922 - val_recall: 0.0922
Epoch 4/20
266/266 [==============================] - 121s - loss: 14.4820 - binary_accuracy: 0.9982 - fmeasure: 0.1015 - precisi

In [75]:
#freeze layers
for layer in model.layers[:15]:
    layer.trainable = False
    print(layer)
print("")
for layer in model.layers[15:]:
    layer.trainable = True
    print(layer)

In [76]:
model.fit(tr_sets,tr_vals,
          nb_epoch=20,
          verbose=1,
          batch_size = 32,
          validation_data=(val_sets, val_vals),
         shuffle= True)

Train on 266 samples, validate on 141 samples
Epoch 1/20
266/266 [==============================] - 135s - loss: 14.4820 - binary_accuracy: 0.9982 - fmeasure: 0.1015 - precision: 0.1015 - recall: 0.1015 - val_loss: 14.6320 - val_binary_accuracy: 0.9982 - val_fmeasure: 0.0922 - val_precision: 0.0922 - val_recall: 0.0922
Epoch 2/20
266/266 [==============================] - 135s - loss: 14.4820 - binary_accuracy: 0.9982 - fmeasure: 0.1015 - precision: 0.1015 - recall: 0.1015 - val_loss: 14.6320 - val_binary_accuracy: 0.9982 - val_fmeasure: 0.0922 - val_precision: 0.0922 - val_recall: 0.0922
Epoch 3/20
266/266 [==============================] - 133s - loss: 14.4820 - binary_accuracy: 0.9982 - fmeasure: 0.1015 - precision: 0.1015 - recall: 0.1015 - val_loss: 14.6320 - val_binary_accuracy: 0.9982 - val_fmeasure: 0.0922 - val_precision: 0.0922 - val_recall: 0.0922
Epoch 4/20
266/266 [==============================] - 134s - loss: 14.4820 - binary_accuracy: 0.9982 - fmeasure: 0.1015 - precisi

## Predictions

In [61]:
a = np.array([val_sets[27]])
preds = model.predict(a)
np.shape(a)

(1, 3, 224, 224)

Formating the output

In [68]:
print('Predicted:', decode_predictions(np.array([val_vals[0],val_vals[1]])))
# print: [[u'n02504458', u'African_elephant']]

Predicted: [[('n01981276', 'king_crab', 1.0), ('n15075141', 'toilet_tissue', 0.0), ('n02319095', 'sea_urchin', 0.0), ('n02395406', 'hog', 0.0), ('n02391049', 'zebra', 0.0)], [('n01443537', 'goldfish', 1.0), ('n15075141', 'toilet_tissue', 0.0), ('n02319095', 'sea_urchin', 0.0), ('n02395406', 'hog', 0.0), ('n02391049', 'zebra', 0.0)]]


In [63]:
np.shape(preds)

(1, 1000)

In [66]:
np.shape([val_vals[0],val_vals[1]])

(2, 1000)